In [1]:
import math
import time
import random

In [2]:
pessoas = [('Amanda', 'CWB'),
           ('Pedro', 'GIG'),
           ('Marcos', 'POA'),
           ('Priscila', 'FLN'),
           ('Jessica', 'CNF'),
           ('Paulo', 'GYN')]

destino = 'GRU'

In [3]:
voos = {}

for l in open('voos.txt'):
    origem, dest, saida, chegada, preco = l.split(',')
    voos.setdefault((origem, dest), [])
    voos[(origem, dest)].append((saida, chegada, int(preco)))

In [4]:
def print_agenda(agenda):
    for idx, voo in enumerate(agenda):
        nome = pessoas[idx][0]
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        print("%10s%10s %5s-%5s R$%3s %5s-%5s R$%3s" % (nome, origem, ida[0], ida[1], ida[2],
                                                        volta[0], volta[1], volta[2]))

agenda = [(1,3), (3, 2), (7,3), (6,3), (2,4), (5,3)]

print_agenda(agenda)

    Amanda       CWB  8:04-10:11 R$ 95 10:33-12:03 R$ 74
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 17:08-19:08 R$262 10:32-13:16 R$139
  Priscila       FLN 15:34-18:11 R$326 11:08-14:38 R$262
   Jessica       CNF  9:42-11:32 R$169 12:08-14:47 R$231
     Paulo       GYN 13:37-15:08 R$250 11:07-13:24 R$171


In [5]:
def get_minutos(hora:str):
    tempo = time.strptime(hora, '%H:%M')
    return tempo[3] * 60 + tempo[4]

get_minutos('1:10')

70

In [6]:
def custo(solucao:list):
    preco_total = 0
    # menor possível
    ultima_chegada = 0
    # maior possível
    primeira_partida = 1439 #23:59 em minutos
    total_espera = 0

    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        preco_total += ida[2]
        preco_total += volta[2]

        # atualiza a ultima chegada ao aeroporto
        if ultima_chegada < get_minutos(ida[1]):
            ultima_chegada = get_minutos(ida[1])

        # atualiza com a primeira chegada ao aeroporto
        if primeira_partida > get_minutos(volta[0]):
            primeira_partida = get_minutos(volta[0])
        
    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        # calcula o tanto de espera no aeroporto
        total_espera += ultima_chegada - get_minutos(ida[1])
        total_espera += get_minutos(volta[0]) - primeira_partida

    # se virou um dia, adiciona penalidade em dinheiro
    if ultima_chegada > primeira_partida:
        preco_total += 50

    return preco_total + total_espera

In [7]:
custo(agenda)

4472